In [8]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [9]:
!pip install transformers
!pip install datasets
!pip install audiomentations
!pip install evaluate
!pip install jiwer
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 1.5 MB/s eta 0:00:00


# Fine-tune Whisper

## Prepare Data and Model

### Load Model

In [10]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

(…)i/whisper-small/resolve/main/config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

(…)mall/resolve/main/generation_config.json:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

In [11]:
# include forced token in the training
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
# to use gradient checkpointing
model.config.use_cache = False

### Load Datasets

In [12]:
AUDIO_COLUMN_NAME = "audio"
TEXT_COLUMN_NAME = "sentence"

In [13]:
from datasets import Audio, DatasetDict, concatenate_datasets, load_dataset

def normalize_dataset(ds, audio_column_name=None, text_column_name=None):
    if audio_column_name is not None and audio_column_name != AUDIO_COLUMN_NAME:
        ds = ds.rename_column(audio_column_name, AUDIO_COLUMN_NAME)
    if text_column_name is not None and text_column_name != TEXT_COLUMN_NAME:
        ds = ds.rename_column(text_column_name, TEXT_COLUMN_NAME)
    # resample to the same sampling rate
    ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
    # normalise columns to ["audio", "sentence"]
    ds = ds.remove_columns(set(ds.features.keys()) - set([AUDIO_COLUMN_NAME, TEXT_COLUMN_NAME]))
    return ds

In [14]:
raw_datasets = DatasetDict()

ds_train_mcv = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="train+validation")
ds_train_mcv = normalize_dataset(ds_train_mcv)

#ds_train_mls = load_dataset("facebook/multilingual_librispeech", "french", split="train+validation")
#ds_train_mls = normalize_dataset(ds_train_mls, text_column_name="text")

#raw_datasets["train"] = concatenate_datasets([ds_train_mcv, ds_train_mls])
raw_datasets["train"] = concatenate_datasets([ds_train_mcv])


# NB: shuffle concatenated dataset
raw_datasets["train"] = raw_datasets["train"].shuffle(seed=10)

raw_datasets["eval"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test")
raw_datasets["eval"] = normalize_dataset(raw_datasets["eval"])

raw_datasets

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]


Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 4361it [00:00, 26738.04it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...

Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 3328it [00:00, 77598.46it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 680it [00:00, 56668.52it/s]


DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 6540
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 2894
    })
})

In [15]:
# only use part of training and test set for demonstration
# comment this
num_train_examples = 500
num_test_examples = 100

raw_datasets["train"] = raw_datasets["train"].select(range(num_train_examples))
raw_datasets["eval"] = raw_datasets["eval"].select(range(num_test_examples))

raw_datasets

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 500
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 100
    })
})

### Data Augmentation

In [17]:
# download musan dataset for background noise
! wget https://www.openslr.org/resources/17/musan.tar.gz
! tar -zxvf musan.tar.gz
! rm -r ./musan/speech

--2023-11-04 21:40:10--  https://www.openslr.org/resources/17/musan.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/17/musan.tar.gz [following]
--2023-11-04 21:40:10--  http://us.openslr.org/resources/17/musan.tar.gz
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11086114085 (10G) [application/x-gzip]
Saving to: ‘musan.tar.gz’

musan.tar.gz        100%[===================>]  10.32G  35.0MB/s    in 5m 7s   

2023-11-04 21:45:18 (34.4 MB/s) - ‘musan.tar.gz’ saved [11086114085/11086114085]

musan/
musan/music/
musan/music/fma/
musan/music/fma/music-fma-0001.wav
musan/music/fma/music-fma-0002.wav
musan/music/fma/music-fma-0003.wav
musan/music/fma/music-fma-0

In [18]:
from audiomentations import (
    AddBackgroundNoise,
    AddGaussianNoise,
    Compose,
    Gain,
    OneOf,
    PitchShift,
    PolarityInversion,
    TimeStretch,
)

musan_dir = "./musan"

# define augmentation
augmentation = Compose(
    [
        TimeStretch(min_rate=0.9, max_rate=1.1, p=0.2, leave_length_unchanged=False),
        Gain(min_gain_in_db=-6, max_gain_in_db=6, p=0.1),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.2),
        OneOf(
            [
                AddBackgroundNoise(
                    sounds_path=musan_dir, min_snr_in_db=1.0, max_snr_in_db=5.0, noise_transform=PolarityInversion(), p=1.0
                ),
                AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.015, p=1.0),
            ],
            p=0.2,
        ),
    ]
)


def augment_dataset(batch):
    # load and (possibly) resample audio data to 16kHz
    sample = batch[AUDIO_COLUMN_NAME]

    # apply augmentation
    augmented_waveform = augmentation(sample["array"], sample_rate=sample["sampling_rate"])
    batch[AUDIO_COLUMN_NAME]["array"] = augmented_waveform
    return batch

In [19]:
preprocessing_num_workers = 4

In [20]:
# augment training data
augmented_raw_training_dataset = raw_datasets["train"].map(
    augment_dataset, num_proc=preprocessing_num_workers, desc="augment train dataset"
)

# combine
raw_datasets["train"] = concatenate_datasets([raw_datasets["train"], augmented_raw_training_dataset])
raw_datasets["train"] = raw_datasets["train"].shuffle(seed=10)

raw_datasets

/usr/local/lib/python3.10/dist-packages/audiomentations/augmentations/add_background_noise.py:239: UserWarning: Warning: the LRU cache of AddBackgroundNoise gets discarded when pickling it. E.g. this means the cache will not be used when using AddBackgroundNoise together with multiprocessing on Windows
  warnings.warn(


augment train dataset (num_proc=4):   0%|          | 0/500 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/audiomentations/core/transforms_interface.py:61: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/audiomentations/core

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1000
    })
    eval: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 100
    })
})

### Preprocessing

In [21]:
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

normalizer = BasicTextNormalizer()

In [23]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-medium", language="hi", task="transcribe")

In [24]:
do_normalize_text = True


def prepare_dataset(batch):
    # load
    audio = batch[AUDIO_COLUMN_NAME]
    # compute log-Mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[
        0
    ]
    # compute input length of audio sample in seconds
    batch["input_length"] = len(audio["array"]) / audio["sampling_rate"]

    # process targets
    input_str = normalizer(batch[TEXT_COLUMN_NAME]).strip() if do_normalize_text else batch[TEXT_COLUMN_NAME]
    # encode target text to label ids
    batch["labels"] = processor.tokenizer(input_str).input_ids

    return batch

In [25]:
vectorized_datasets = raw_datasets.map(
    prepare_dataset, remove_columns=next(iter(raw_datasets.values())).column_names, desc="preprocess dataset"
)

preprocess dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

preprocess dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

### Remove Long Audio

In [26]:
max_input_length = 30
min_input_length = 0


def is_audio_in_length_range(length):
    return length > min_input_length and length < max_input_length

In [27]:
vectorized_datasets = vectorized_datasets.filter(
    is_audio_in_length_range, num_proc=preprocessing_num_workers, input_columns=["input_length"]
)

vectorized_datasets

Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 1000
    })
    eval: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 100
    })
})

### Remove Long Text

In [28]:
max_label_length = model.config.max_length
max_label_length

448

In [29]:
def is_labels_in_length_range(labels):
    return len(labels) < max_label_length

In [30]:
vectorized_datasets = vectorized_datasets.filter(
    is_labels_in_length_range, num_proc=preprocessing_num_workers, input_columns=["labels"]
)

vectorized_datasets

Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 1000
    })
    eval: Dataset({
        features: ['input_features', 'input_length', 'labels'],
        num_rows: 100
    })
})

## Training and Evaluation

### Data Collator

In [31]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

import torch


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        # convert to tensors
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad label ids to the max length in the batch
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [32]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

In [33]:
import evaluate

metric = evaluate.load("wer")

In [34]:
# evaluate with the 'normalised' WER
do_normalize_eval = True


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    if do_normalize_eval:
        pred_str = [normalizer(pred) for pred in pred_str]
        # perhaps already normalised
        label_str = [normalizer(label) for label in label_str]
        # filtering step to only evaluate the samples that correspond to non-zero references
        pred_str = [pred_str[i] for i in range(len(pred_str)) if len(label_str[i]) > 0]
        label_str = [label_str[i] for i in range(len(label_str)) if len(label_str[i]) > 0]

    wer = metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Training Configuration

In [38]:
!pip install accelerate -U

In [40]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.24.1
aiohttp                          3.8.6
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audiomentations                  0.33.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.13.1
backcall                         0.2.0
b

In [39]:
from transformers import Seq2SeqTrainingArguments
import torch

training_args = Seq2SeqTrainingArguments(
    output_dir="./outputs/whisper_medium_ft",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    warmup_steps=800,
    max_steps=8000,
    learning_rate=6.25e-6,
    weight_decay=0.01,
    gradient_checkpointing=True,
    fp16=True,
    predict_with_generate=True,
    generation_max_length=225,
    logging_steps=25,
    report_to=["tensorboard"],
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

ImportError: ignored

### Initialize Trainer

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=vectorized_datasets["train"],
    eval_dataset=vectorized_datasets["eval"],
    tokenizer=processor,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

### Training

In [ ]:
train_result = trainer.train()

In [ ]:
model.save_pretrained(training_args.output_dir)
processor.save_pretrained(training_args.output_dir)

## Evaluation

In [ ]:
eval_metrics = trainer.evaluate(metric_key_prefix="eval")

eval_metrics

# Predict

In [ ]:
model

In [ ]:
# Source https://thepythoncode.com/article/speech-recognition-using-huggingface-transformers-in-python

import torchaudio
device = "cuda:0" if torch.cuda.is_available() else "cpu"

def load_audio(audio_path):
  """Load the audio file & convert to 16,000 sampling rate"""
  # load our wav file
  speech, sr = torchaudio.load(audio_path)
  resampler = torchaudio.transforms.Resample(sr, 16000)
  speech = resampler(speech)
  return speech.squeeze()


def get_transcription_whisper(audio_path, model, processor, language="english", skip_special_tokens=True):
  # resample from whatever the audio sampling rate to 16000
  speech = load_audio(audio_path)
  # get the input features
  input_features = processor(speech, return_tensors="pt", sampling_rate=16000).input_features.to(device)
  # get special decoder tokens for the language
  forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task="transcribe")
  # perform inference
  predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
  # decode the IDs to text
  transcription = processor.batch_decode(predicted_ids, skip_special_tokens=skip_special_tokens)[0]
  return transcription

In [ ]:
# Arabic transcription, should be crap!
arabic_transcription = get_transcription_whisper("https://datasets-server.huggingface.co/assets/arabic_speech_corpus/--/clean/train/0/audio/audio.wav",
                          model,
                          processor,
                          language="arabic",
                          skip_special_tokens=True)
arabic_transcription